In [3]:
import mhcflurry as mhc

import sklearn
import numpy
import scipy

Using Theano backend.
/Users/carlomazzaferro/anaconda/envs/MHC_NN/lib/python3.5/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/carlomazzaferro/.matplotlib/matplotlibrc", line #2
  (fname, cnt))
/Users/carlomazzaferro/anaconda/envs/MHC_NN/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
train_data = kim2014_train.get_allele("HLA-A3301")
new_model = mhc.class1_allele_specific.Class1BindingPredictor()
new_model.fit_dataset(train_data)

In [6]:

test_data = kim2014_test.get_allele("HLA-A3301")
predictions = new_model.predict(test_data.peptides)

In [10]:
train_data.peptides

3040

In [2]:
ds = '/Users/carlomazzaferro/Desktop/combined_human_class1_dataset.csv'

kim2014 = mhc.dataset.Dataset.from_csv(ds)

In [4]:
kim2014_full = mhc.dataset.Dataset.from_csv(
    mhc.downloads.get_path("data_kim2014", "bdata.20130222.mhci.public.1.txt"))

kim2014_train = mhc.dataset.Dataset.from_csv(
    mhc.downloads.get_path("data_kim2014", "bdata.2009.mhci.public.1.txt"))
kim2014_test = mhc.dataset.Dataset.from_csv(
    mhc.downloads.get_path("data_kim2014", "bdata.2013.mhci.public.blind.1.txt"))

len(kim2014_full), len(kim2014_train), len(kim2014_test)

(179692, 137654, 27680)

In [4]:
train = kim2014_train.get_allele("HLA-A0101")
len(train)

3725

In [5]:
X_index, kmer_affinities, combined_sample_weights, original_peptide_indices = train.kmer_index_encoding()

In [6]:
def ic50_to_regression_target(ic50, max_ic50=50000):

    log_ic50 = numpy.log(ic50) / numpy.log(max_ic50)
    regression_target = 1.0 - log_ic50
    # clamp to values between 0, 1
    regression_target = numpy.maximum(regression_target, 0.0)
    regression_target = numpy.minimum(regression_target, 1.0)
    return regression_target

def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500, max_ic50=50000):

    y_pred = ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(
            ic50_y <= threshold_nm,
            y_pred,
            sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(
            ic50_y <= threshold_nm,
            ic50_y_pred <= threshold_nm,
            sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan

    return dict(
        auc=auc,
        f1=f1,
        tau=tau)
Y_combined = ic50_to_regression_target(kmer_affinities, max_ic50=50000)

In [1]:
import mhcflurry as mhc
from sklearn.model_selection import train_test_split
import sklearn
import numpy
import scipy
import tflearn

def regression_target_to_ic50(y, max_ic50=50000.0):
    return max_ic50 ** (1.0 - y)

def ic50_to_regression_target(ic50, max_ic50=50000.0):

    log_ic50 = numpy.log(ic50) / numpy.log(max_ic50)
    regression_target = 1.0 - log_ic50
    # clamp to values between 0, 1
    regression_target = numpy.maximum(regression_target, 0.0)
    regression_target = numpy.minimum(regression_target, 1.0)
    return regression_target

kim2014_full = mhc.dataset.Dataset.from_csv(mhc.downloads.get_path("data_kim2014", "bdata.20130222.mhci.public.1.txt"))

kim2014_train = mhc.dataset.Dataset.from_csv(mhc.downloads.get_path("data_kim2014", "bdata.2009.mhci.public.1.txt"))
kim2014_test = mhc.dataset.Dataset.from_csv(mhc.downloads.get_path("data_kim2014", "bdata.2013.mhci.public.blind.1.txt"))

train = kim2014_train.get_allele("HLA-A0101")

X_index, kmer_affinities, combined_sample_weights, original_peptide_indices = train.kmer_index_encoding()
Y_combined = ic50_to_regression_target(kmer_affinities, max_ic50=50000)

xTr, xTe, yTr, yTe = train_test_split(X_index, Y_combined)

yTr = numpy.reshape(yTr, (yTr.shape[0], 1))
yTe = numpy.reshape(yTe, (yTe.shape[0], 1))

Using Theano backend.
/Users/carlomazzaferro/anaconda/envs/MHC_NN/lib/python3.5/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/carlomazzaferro/.matplotlib/matplotlibrc", line #2
  (fname, cnt))
/Users/carlomazzaferro/anaconda/envs/MHC_NN/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print(xTr.shape, xTe.shape, yTr.shape, yTe.shape)

(6520, 9) (2174, 9) (6520, 1) (2174, 1)


In [3]:
import tflearn
import tensorflow as tf 

x = tf.placeholder(shape=(None, 9), dtype=tf.float32)
y_ = tf.placeholder(shape=(None, 1, dtype=tf.float32)
                    
net = tflearn.input_data(placeholder=x)
net = tflearn.embedding(net, input_dim=21, output_dim=32, weights_init='xavier')
net = tflearn.fully_connected(net, 100, activation='tanh')
net = tflearn.layers.normalization.batch_normalization(net)
net = tflearn.dropout(net, 0.1)
net = tflearn.fully_connected(net, 1, activation='sigmoid')

# net = tflearn.batch_normalization()
rmsprop = tflearn.RMSProp(learning_rate=0.0001, decay=0.99)
net = tflearn.regression(net, optimizer=rmsprop,
                         loss='mean_square')

# Training
model = tflearn.DNN(net, tensorboard_verbose=1)
model.fit(xTr, yTr, n_epoch=40, validation_set=(xTe, yTe), show_metric=True,
          batch_size=70)
#tflearn.is_training(False)
ps = model.predict(xTe)


Training Step: 3759  | total loss: 0.05826 | time: 2.235s
| RMSProp | epoch: 040 | loss: 0.05826 - binary_acc: 0.0045 -- iter: 6510/6520
Training Step: 3760  | total loss: 0.05991 | time: 3.263s
| RMSProp | epoch: 040 | loss: 0.05991 - binary_acc: 0.0040 | val_loss: 0.04982 - val_acc: 0.0023 -- iter: 6520/6520
--


In [11]:
numpy.max(xTe)

20

In [5]:
import pandas
ps = model.predict(xTe)
preds = [regression_target_to_ic50(i[0]) for i in ps]
targs = [regression_target_to_ic50(i[0]) for i in yTe]
pandas.DataFrame([preds, targs]).T


,0,1
0,13779.433073,20000.000000
1,9219.578326,50000.000000
2,986.525863,3106.005166
3,1236.310011,50000.000000
4,3050.660671,20000.000000
5,1948.502773,20000.000000
6,11.198050,33.000000
7,41.551778,1194.731086
8,4758.773243,13633.749813
9,29.460669,22.629947
